In [1]:
import torch
def multiplication_mod_p_data(p, eq_token, op_token):
    """x◦y = x/y (mod p) for 0 ≤ x < p, 0 < y < p
    """
    x = torch.arange(p)
    y = torch.arange(1, p)
    x, y = torch.cartesian_prod(x, y).T

    eq = torch.ones_like(x) * eq_token
    op = torch.ones_like(x) * op_token
    result = x * y % p

    # "All of our experiments used a small transformer trained on datasets of
    # equations of the form a◦b = c, where each of “a”, “◦”, “b”, “=”, and “c”
    # is a seperate token"
    return torch.stack([x, op, y, eq, result])

from argparse import ArgumentParser
parser = ArgumentParser()
parser.add_argument("--p", type=int, default=97)
args = parser.parse_args('')

eq_token = args.p
op_token = args.p + 1

data = multiplication_mod_p_data(args.p, eq_token, op_token)

train_idx, valid_idx = torch.randperm(data.shape[1]).split(data.shape[1] // 2)
train_data, valid_data = data[:, train_idx], data[:, valid_idx]

In [2]:
# dataset_train, dataset_test = create_mnist_dataset("./data")


In [30]:
import torch
from torch.utils.data import Dataset, DataLoader, Sampler
import numpy as np

class ColumnDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
        self.num_columns = data.shape[1]

    def __len__(self):
        return self.num_columns

    def __getitem__(self, index):
        return self.data[:, index]
    
# Custom sampler that samples per column
class ColumnSampler(Sampler):
    def __init__(self, data_source):
        self.data_source = data_source

    def __iter__(self):
        # Return an iterator over the columns (indices 0 to number of columns - 1)
        return iter(range(len(self.data_source)))

    def __len__(self):
        return len(self.data_source)

In [39]:
import torch
from torch.utils.data import Dataset, DataLoader, Sampler

# Dataset where each column is a data point
class ColumnDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
        self.num_columns = data.shape[1]

    def __len__(self):
        return self.num_columns

    def __getitem__(self, index):
        return self.data[:, index]

# Custom sampler that samples columns
class ColumnSampler(Sampler):
    def __init__(self, data_source, batch_size):
        self.data_source = data_source
        self.batch_size = batch_size
        self.num_columns = len(data_source)

    def __iter__(self):
        indices = torch.randperm(self.num_columns).tolist()
        # Yield indices in batches
        for i in range(0, self.num_columns, self.batch_size):
            yield indices[i:i + self.batch_size]

    def __len__(self):
        return (self.num_columns + self.batch_size - 1) // self.batch_size

# Modified custom collate function to collapse the second dimension
def custom_collate_fn(batch):
    batch = torch.stack(batch, dim=1)  # Stack along dimension 1
    return batch.squeeze(1)  # Remove the singleton dimension

# Example usage
if __name__ == "__main__":
    # Example data with shape (5, 4656)
    data = torch.randn(5, 4656)




torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 32])


In [36]:
train_data.shape

torch.Size([5, 4656])

In [40]:
# Algorithmic data
# dataset_train = train_data.T
# dataset_test = valid_data.T

dataset_train = ColumnDataset(train_data)
dataset_test = ColumnDataset(valid_data)

batch_size = 32  # Adjust as needed
    
# Create the custom ColumnSampler
sampler_train = ColumnSampler(dataset_train, batch_size=batch_size)
sampler_test = ColumnSampler(dataset_test, batch_size=batch_size)
    
# Create a DataLoader with the custom sampler and custom collate function
train_loader = DataLoader(dataset_train, sampler=sampler_train, collate_fn=custom_collate_fn)
test_loader = DataLoader(dataset_test, sampler=sampler_test, collate_fn=custom_collate_fn)


In [41]:
next(iter(train_loader)).shape

torch.Size([5, 32])

In [42]:
from main import Decoder
# device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = Decoder(
    dim=128, num_layers=2, num_heads=4, num_tokens=args.p + 2, seq_len=5
).to(device)


In [ ]:
import torch
results = torch.load('results/res_test_none.pt')
# results = torch.load('results/res_test_ma_w100_l5_wd10e-02.pt')
# results.keys()

In [36]:
weights = results['net']
model.load_state_dict(weights[0])

<All keys matched successfully>

In [43]:
import torch
import torch.nn.functional as F
from torch_influence import BaseObjective

class MyObjective(BaseObjective):

    def train_outputs(self, model, batch):
        return model(batch[0])

    def train_loss_on_outputs(self, outputs, batch):
        return F.cross_entropy(outputs, batch[1])  # mean reduction required

    def train_regularization(self, params):
        return 0.01 * torch.square(params.norm())

    # training loss by default taken to be 
    # train_loss_on_outputs + train_regularization

    def test_loss(self, model, params, batch):
        return F.cross_entropy(model(batch[0]), batch[1])  # no regularization in test loss

In [52]:
train_batch = next(iter(train_loader)).to(device)
train_batch.shape

torch.Size([5, 32])

Decoder(
  (token_embeddings): Embedding(99, 128)
  (position_embeddings): Embedding(5, 128)
  (layers): ModuleList(
    (0-1): 2 x Block(
      (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=512, out_features=128, bias=True)
      )
    )
  )
  (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_features=128, out_features=99, bias=False)
)

In [54]:
from torch_influence import AutogradInfluenceModule
   
module = AutogradInfluenceModule(
    model=model,
    objective=MyObjective(),  
    train_loader=train_loader,
    test_loader=test_loader,
    device=device,
    damp=0.001
)

# influence scores of training points 1, 2, and 3 on test point 0
scores = module.influences([1, 2, 3], [0])

RuntimeError: torch.cat(): expected a non-empty list of Tensors